In [18]:
# Bibliotecas necessárias para pegar as informações da wikipedia
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Página com os dividendos
stock_url = 'https://statusinvest.com.br/acoes/abev3'

# Fazendo o resquest da página
response = requests.get(stock_url)
soup = BeautifulSoup(response.text,'html.parser')

# Extraindo apenas a tabela
tabela_html = soup.find('table')

# Transformando a tabela em um data frame
tabela = pd.read_html(str(tabela_html),decimal=',', thousands='.')
dados_dividendos = tabela[0]

# Mostrando as 10 primeiras linhas da tabela
dados_dividendos.head(10)

,Tipo,DATA COM,Pagamento,Valor
0,Dividendo,17/12/2021,30/12/2021,0.1334
1,JCP,17/12/2021,30/12/2021,0.4702
2,Dividendo,13/01/2021,28/01/2021,0.0767
3,JCP,17/12/2020,30/12/2020,0.4137
4,JCP,19/12/2019,30/12/2019,0.4906
5,JCP,18/12/2018,28/12/2018,0.3200
6,Dividendo,15/06/2018,30/07/2018,0.1600
7,Dividendo,31/01/2018,22/02/2018,0.0700
8,JCP,18/12/2017,28/12/2017,0.3100
9,Dividendo,23/06/2017,17/07/2017,0.1600


In [27]:
# Tranformando em função
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Página com os dividendos
def getDividendData(ticker,init_date,end_date):
    stock_url = 'https://statusinvest.com.br/acoes/'+ticker

    # Fazendo o resquest da página
    response = requests.get(stock_url)
    soup = BeautifulSoup(response.text,'html.parser')

    # Extraindo apenas a tabela
    tabela_html = soup.find('table')

    # Transformando a tabela em um data frame
    tabela = pd.read_html(str(tabela_html),decimal=',', thousands='.')
    dados_dividendos = tabela[0]
    
    # Convertendo para data
    dados_dividendos['Pagamento'] = pd.to_datetime(dados_dividendos['Pagamento'],format='%d/%m/%Y',errors='coerce')
    dados_dividendos['DATA COM'] = pd.to_datetime(dados_dividendos['DATA COM'],format='%d/%m/%Y',errors='coerce')

    # Filtrando no período desejado
    dados_dividendos_ano = dados_dividendos[(dados_dividendos['DATA COM'] > init_date) 
                                            & (dados_dividendos['DATA COM'] < end_date)]
    
    return dados_dividendos_ano[dados_dividendos_ano['Pagamento'].notna()]

# Teste
ticker = 'cple6'
dados_dividendos = getDividendData(ticker,'2021-05-21','2022-01-01')
dados_dividendos

,Tipo,DATA COM,Pagamento,Valor
2,JCP,2021-09-30,2021-11-30,0.09074606
3,Dividendo,2021-09-30,2021-11-30,0.45328533


In [28]:
def formatarDadosDividendos(dados_dividendos,numero_de_acoes,ticker):

    # Dividendos -> Para ser declarado na ficha 'Rendimentos Isentos e Não Tributáveis”,
    # no campo “09 – Lucros e dividendos recebidos”.
    
    dados_dividendos_separados = dados_dividendos[dados_dividendos.Tipo == 'Dividendo']

    dividendos_total_no_periodo = dados_dividendos_separados.Valor.astype(float).sum()

    print('Dividendos recebidos R$'+
                      str(round(dividendos_total_no_periodo*numero_de_acoes,2)).replace('.',','))

#Teste
numero_de_acoes = 1
formatarDadosDividendos(dados_dividendos,numero_de_acoes,ticker)

Dividendos recebidos R$0,45


In [29]:
def formatarDadosJCP(dados_dividendos,numero_de_acoes,ticker):

    # JCP para serem declarados em “Rendimentos Sujeitos à Tributação Exclusiva/Definitiva”,
    # na opção “10 – Juros sobre capital próprio”.
    
    dados_jcp = dados_dividendos[dados_dividendos.Tipo == 'JCP']

    jcp_total_no_periodo = dados_jcp.Valor.astype(float).sum()

    print('Juros sob capital próprio recebidos: R$'+
                      str(round(jcp_total_no_periodo*numero_de_acoes,2)).replace('.',','))

#Teste
numero_de_acoes = 1
formatarDadosJCP(dados_dividendos,numero_de_acoes,ticker)

Juros sob capital próprio recebidos: R$0,09


# Pegando dados adicionais da empresa

In [30]:
# Página com os dados da empresa
dados_url = 'https://orealvalor.com.br/blog/cnpj-de-todas-as-acoes-da-b3/'

# Fazendo o resquest da página
response = requests.get(dados_url)
soup = BeautifulSoup(response.text,'html.parser')

# Extraindo apenas a tabela
tabela_html = soup.find('table')

# Transformando a tabela em um data frame
tabela = pd.read_html(str(tabela_html),decimal=',', thousands='.')
dados = tabela[0]

# Convertendo os ticker para minusculo
dados['Ticker'] = dados['Ticker'].str.lower()

# Mostrando as 10 primeiras linhas da tabela
dados.head(10)

,Ticker,CNPJ,Razão Social
0,aalr3,42.771.949/0001-35,CENTRO DE IMAGEM DIAGNÓSTICOS S.A.
1,abcb4,28.195.667/0001-06,BANCO ABC BRASIL S/A
2,abev3,07.526.557/0001-00,AMBEV S.A.
3,aeri3,12.528.708/0001-07,Aeris Indústria e Com. de Equip. para Geração ...
4,aesb3,37.663.076/0001-07,AES BRASIL ENERGIA S.A.
5,aflt3,10.338.320/0001-00,AFLUENTE TRANSMISSÃO DE ENERGIA ELETRICA S/A
6,agro3,07.628.528/0001-59,BRASILAGRO CIA BRAS DE PROP AGRICOLAS
7,agxy3,21.240.146/0001-84,AGROGALAXY PARTICIPACOES S.A.
8,aheb3,62.002.886/0001-60,SÃO PAULO TURISMO S. A.
9,aheb5,62.002.886/0001-60,SÃO PAULO TURISMO S. A.


In [31]:
def getCompanyData(ticker):

    # Página com os dados da empresa
    dados_url = 'https://orealvalor.com.br/blog/cnpj-de-todas-as-acoes-da-b3/'

    # Fazendo o resquest da página
    response = requests.get(dados_url)
    soup = BeautifulSoup(response.text,'html.parser')

    # Extraindo apenas a tabela
    tabela_html = soup.find('table')

    # Transformando a tabela em um data frame
    tabela = pd.read_html(str(tabela_html),decimal=',', thousands='.')
    dados = tabela[0]

    # Convertendo os ticker para minusculo
    dados['Ticker'] = dados['Ticker'].str.lower()
    
    # Filtrando pelo ticker
    dados_do_ticker = dados[dados['Ticker'] == ticker]

    # Imprimindo os dados que preciso
    print('CNPJ: '+ dados_do_ticker.iloc[0].at['CNPJ'])
    print('Nome: ' + dados_do_ticker.iloc[0].at['Razão Social'])

In [32]:
## Fazendo agora uma função que me retorne tudo junto

# Pegar os dados
def declaracao(dados_acao):
    dados_dividendos = getDividendData(dados_acao['ticker'],dados_acao['data_compra'],dados_acao['data_fim'])
    
    # Dados da empresa
    getCompanyData(dados_acao['ticker'])
    
    formatarDadosDividendos(dados_dividendos,dados_acao['numero_de_acoes'],dados_acao['ticker'])
    formatarDadosJCP(dados_dividendos,dados_acao['numero_de_acoes'],dados_acao['ticker'])
    

# dados_acao = {'ticker':,'data_compra',]
dados_acao = {'ticker':'b3sa3',
              'data_compra':'2021-05-21',
              'data_fim': '2022-01-01',
              'numero_de_acoes': 28}

declaracao(dados_acao)


CNPJ: 09.346.601/0001-25
Nome: B3 S.A.
Dividendos recebidos R$13,11
Juros sob capital próprio recebidos: R$3,92


In [87]:
dados_acao = {'ticker':'cple6',
              'data_compra':'2021-05-21',
              'data_fim': '2022-01-01',
              'numero_de_acoes': 82}

declaracao(dados_acao)

CNPJ: 76.483.817/0001-20
Nome: COMPANHIA PARANAENSE DE ENERGIA
Dividendos recebidos R$37,17
Juros sob capital próprio recebidos: R$7,44


In [88]:
dados_acao = {'ticker':'agro3',
              'data_compra':'2021-05-21',
              'data_fim': '2022-01-01',
              'numero_de_acoes': 16}

declaracao(dados_acao)

CNPJ: 07.628.528/0001-59
Nome: BRASILAGRO CIA BRAS DE PROP AGRICOLAS
Dividendos recebidos R$41,94
Juros sob capital próprio recebidos: R$0,0


In [89]:
dados_acao = {'ticker':'taee11',
              'data_compra':'2021-05-21',
              'data_fim': '2022-01-01',
              'numero_de_acoes': 12}

declaracao(dados_acao)

CNPJ: 07.859.971/0001-30
Nome: TRANSMISSORA ALIANÇA DE ENERGIA ELÉTRICA S.A.
Dividendos recebidos R$11,18
Juros sob capital próprio recebidos: R$7,04


In [90]:
dados_acao = {'ticker':'abev3',
              'data_compra':'2021-05-21',
              'data_fim': '2022-01-01',
              'numero_de_acoes': 34}

declaracao(dados_acao)

CNPJ: 07.526.557/0001-00
Nome: AMBEV S.A.
Dividendos recebidos R$4,54
Juros sob capital próprio recebidos: R$15,99


# Testes

In [28]:
getDividendData(dados_acao['ticker'],dados_acao['data_compra'],dados_acao['data_fim'])

,Tipo,DATA COM,Pagamento,Valor
0,Dividendo,2021-12-17,2021-12-30,0.1334
1,JCP,2021-12-17,2021-12-30,0.4702


In [68]:
# Bibliotecas necessárias para pegar as informações da wikipedia
from bs4 import BeautifulSoup
import requests
import pandas as pd


def checkDividends(ticker):

    # Página da wiki com os dados de opopulação de todos os munícipios do Brasil
    stock_url = 'https://statusinvest.com.br/acoes/' + ticker.lower()

    # Fazendo o resquest da página
    response = requests.get(stock_url)
    soup = BeautifulSoup(response.text,'html.parser')

    # Extraindo apenas a tabela
    tabela_html = soup.find('table')

    # Transformando a tabela em um data frame
    tabela = pd.read_html(str(tabela_html),decimal=',', thousands='.')
    dados_dividendos = tabela[0]

    return(dados_dividendos)

checkDividends('GGBR3')

,Tipo,DATA COM,Pagamento,Valor
0,Dividendo,07/03/2022,16/03/2022,0.20
1,Dividendo,05/11/2021,16/11/2021,1.42
2,JCP,05/11/2021,16/11/2021,0.20
3,JCP,27/09/2021,16/11/2021,0.38
4,Dividendo,16/08/2021,26/08/2021,0.54
5,Dividendo,14/05/2021,26/05/2021,0.40
6,JCP,11/03/2021,25/03/2021,0.13
7,JCP,21/12/2020,25/03/2021,0.17
8,Dividendo,06/11/2020,18/11/2020,0.12
9,Dividendo,28/02/2020,11/03/2020,0.03


In [9]:
# Filtrando pelo ticker
dados_do_ticker = dados[dados['Ticker'] == ticker]

# Imprimindo os dados que preciso
print('CNPJ: '+ dados_do_ticker.iloc[0].at['CNPJ'])
print('Nome: ' + dados_do_ticker.iloc[0].at['Razão Social'])

CNPJ: 09.346.601/0001-25
Nome: B3 S.A.


'76.483.817/0001-20'

AttributeError: 'Int64Index' object has no attribute 'int'